In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [3]:
a = pd.read_csv("ev_analysis.csv")
a.head()

,shortDescription,company,vuln_type,vendor,product,theme_keyword,cluster
0,Microsoft MSHTML Framework contains a protecti...,Microsoft MSHTML Framework,NaN,Microsoft,MSHTML Framework,Other,4
1,Microsoft Windows Remote Access Connection Man...,Microsoft Windows Remote Access,NaN,Microsoft,Windows Remote Access,Denial of Service,4
2,Microsoft Windows Shell contains a protection ...,Microsoft Windows Shell,NaN,Microsoft,Windows Shell,Other,4
3,Microsoft Windows Remote Desktop Services cont...,Microsoft Windows Remote Desktop,NaN,Microsoft,Windows Remote Desktop,Other,0
4,Microsoft Desktop Windows Manager contains a t...,Microsoft Desktop Windows Manager,NaN,Microsoft,Desktop Windows Manager,Other,0


In [10]:
# HW

# 1) Vulnerability counts
vuln_counts_df = a['vuln_type'].fillna('Unknown').value_counts().rename_axis('vuln_type').reset_index(name='count')

# 2) Vendor token counts (first token of `company`)
vendor_token = a['company'].fillna('Unknown').apply(lambda x: str(x).split()[0] if str(x)!='nan' else 'Unknown')
vendor_counts_df = vendor_token.value_counts().rename_axis('vendor').reset_index(name='count')

# 3) Heuristic vendor/product split and products per vendor
def split_vendor_product(company):
    if pd.isna(company):
        return (None, None)
    parts = str(company).split()
    if len(parts) == 1:
        return (parts[0], None)
    return (parts[0], ' '.join(parts[1:]))

vp = a['company'].apply(split_vendor_product)
a['vendor'] = vp.apply(lambda x: x[0])
a['product'] = vp.apply(lambda x: x[1])
products_per_vendor_df = a.dropna(subset=['vendor']).groupby('vendor')['product'].nunique().reset_index(name='product_count').sort_values('product_count', ascending=False)

# 4) Theme counts if present
if 'theme_keyword' in a.columns:
    theme_counts_df = a['theme_keyword'].fillna('Other').value_counts().rename_axis('theme').reset_index(name='count')
else:
    theme_counts_df = None

##### Keep results in notebook variables for later plotting
vuln_counts = vuln_counts_df
vendor_counts = vendor_counts_df
products_per_vendor = products_per_vendor_df
theme_counts = theme_counts_df

In [ ]:
## Open
vuln_counts_df.vuln_type.unique()

array(['Remote Code Execution', 'Unknown', 'Privilege Escalation',
       'Denial of Service', 'Command Injection', 'Information Disclosure',
       'Directory Traversal', 'Buffer Overflow', 'Authentication Bypass',
       'a remote attacker to potentially exploit heap corruption via a crafted html page',
       'Cross-Site Scripting', 'SQL Injection', 'an unauthenticated',
       'a remote attacker',
       'a remote attacker to execute code via a crafted html page',
       'an authorized attacker to elevate privileges locally',
       'a remote attacker to execute code inside a sandbox via a crafted html page',
       'for a security feature bypass', 'a remote', 'for code execution',
       'an attacker to escalate privileges and execute http requests on the backend serv',
       'a local attacker to gain system privileges',
       'a remote attacker to exploit heap corruption via a crafted html page',
       'an attacker to escalate privileges',
       'remote attackers to gain priv

In [1]:
### there are some weird ones in vuln_type, let's clean them up --> also there are some words in a sentence such as elevate privileges locally
### that classify as Privilege Escalation